In [ ]:
# === Setup dasar (Step 0 singkat) ===
!pip install -q transformers datasets accelerate evaluate sentencepiece

import torch
from datasets import load_from_disk
from transformers import (
    T5ForConditionalGeneration,
    T5TokenizerFast,
    DataCollatorForSeq2Seq,
    TrainingArguments,
    Trainer,
)

print("CUDA available:", torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.1 MB/s eta 0:00:00


In [ ]:
# === Load tokenized dataset dari Google Drive (hasil Part 1) ===
from google.colab import drive
drive.mount('/content/drive')

load_path = "/content/drive/MyDrive/T5_QA_SQuAD_tokenized"  # samakan dengan Part 1

tokenized_train = load_from_disk(load_path + "/train")
tokenized_valid = load_from_disk(load_path + "/valid")

print(tokenized_train)
print(tokenized_valid)

Mounted at /content/drive
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 5000
})
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1000
})


In [ ]:
model_checkpoint = "t5-base"
tokenizer = T5TokenizerFast.from_pretrained(model_checkpoint)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

In [ ]:
model = T5ForConditionalGeneration.from_pretrained(model_checkpoint)
model.to(device)

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [ ]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
)

# Step 5: TrainingArguments + **Trainer**

In [ ]:
from transformers import (
    T5ForConditionalGeneration,
    T5TokenizerFast,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
)

In [ ]:
batch_size = 4  # kecil dulu biar aman RAM/GPU

training_args = Seq2SeqTrainingArguments(
    output_dir="./t5-base-squad-checkpoints",
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=3e-4,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_steps=100,
    save_total_limit=2,
    predict_with_generate=True,
    fp16=torch.cuda.is_available(),
)

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

/tmp/ipython-input-155155776.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


# **Step 6 training dan save model**

In [ ]:
train_result = trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: raihandiaz01 (raihandiaz01-telkom-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
100,1.447700
200,0.078500
300,0.074300
400,0.066400
500,0.071200
600,0.067900
700,0.070700
800,0.069600
900,0.060500
1000,0.063900


Step,Training Loss
100,1.447700
200,0.078500
300,0.074300
400,0.066400
500,0.071200
600,0.067900
700,0.070700
800,0.069600
900,0.060500
1000,0.063900


In [ ]:
trainer.save_state()          # simpan state trainer (optimizer, scheduler, dll.)
trainer.save_model("./t5-base-squad-finetuned")  # simpan model + tokenizer
tokenizer.save_pretrained("./t5-base-squad-finetuned")

('./t5-base-squad-finetuned/tokenizer_config.json',
 './t5-base-squad-finetuned/special_tokens_map.json',
 './t5-base-squad-finetuned/spiece.model',
 './t5-base-squad-finetuned/added_tokens.json',
 './t5-base-squad-finetuned/tokenizer.json')

In [ ]:
train_result

TrainOutput(global_step=2500, training_loss=0.10195608563423157, metrics={'train_runtime': 954.2295, 'train_samples_per_second': 10.48, 'train_steps_per_second': 2.62, 'total_flos': 6089578905600000.0, 'train_loss': 0.10195608563423157, 'epoch': 2.0})

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

save_dir = "/content/drive/MyDrive/T5_QA_SQuAD_model"

trainer.save_model(save_dir)          # simpan weight + config
tokenizer.save_pretrained(save_dir)   # simpan tokenizer juga

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


('/content/drive/MyDrive/T5_QA_SQuAD_model/tokenizer_config.json',
 '/content/drive/MyDrive/T5_QA_SQuAD_model/special_tokens_map.json',
 '/content/drive/MyDrive/T5_QA_SQuAD_model/spiece.model',
 '/content/drive/MyDrive/T5_QA_SQuAD_model/added_tokens.json',
 '/content/drive/MyDrive/T5_QA_SQuAD_model/tokenizer.json')

In [ ]:
trainer.state.save_to_json(save_dir + "/trainer_state.json")

In [ ]:
from transformers import (
    T5ForConditionalGeneration,
    T5TokenizerFast,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
)

def train_t5_for_epochs(num_epochs, output_dir):
    model = T5ForConditionalGeneration.from_pretrained("t5-base").to(device)
    tokenizer = T5TokenizerFast.from_pretrained("t5-base")

    data_collator = DataCollatorForSeq2Seq(
        tokenizer=tokenizer,
        model=model,
    )

    training_args = Seq2SeqTrainingArguments(
        output_dir=output_dir,
        do_train=True,
        do_eval=True,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        learning_rate=3e-4,
        num_train_epochs=num_epochs,
        weight_decay=0.01,
        logging_steps=100,
        save_total_limit=1,
        predict_with_generate=True,
        fp16=torch.cuda.is_available(),
    )

    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_valid,
        tokenizer=tokenizer,
        data_collator=data_collator,
    )

    trainer.train()
    trainer.save_model(output_dir)
    tokenizer.save_pretrained(output_dir)

    return trainer

In [ ]:
epoch_values = [1, 2, 3]
for ep in epoch_values:
    print(f"\n=== Training {ep} epoch ===")
    out_dir = f"./t5-squad-epochs-{ep}"
    trainer = train_t5_for_epochs(ep, out_dir)


=== Training 1 epoch ===


/tmp/ipython-input-3310871820.py:33: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Step,Training Loss
100,1.448300
200,0.073600
300,0.069100
400,0.064600
500,0.064100
600,0.063100
700,0.065000
800,0.060300
900,0.053700
1000,0.051400



=== Training 2 epoch ===


/tmp/ipython-input-3310871820.py:33: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Step,Training Loss
100,1.447700
200,0.078500
300,0.074300
400,0.066400
500,0.071200
600,0.067900
700,0.070700
800,0.069600
900,0.060500
1000,0.063900



=== Training 3 epoch ===


/tmp/ipython-input-3310871820.py:33: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Step,Training Loss
100,1.446300
200,0.075800
300,0.073200
400,0.069900
500,0.070000
600,0.064900
700,0.076300
800,0.071300
900,0.061600
1000,0.064900


In [ ]:
def build_preprocess_fn(tokenizer, max_input_length=512, max_target_length=32):
    def preprocess_function(examples):
        inputs = []
        targets = []
        for question, context, answers in zip(
            examples["question"],
            examples["context"],
            examples["answers"],
        ):
            input_str = f"question: {question} context: {context}"
            answer_text = answers["text"][0] if len(answers["text"]) > 0 else ""
            inputs.append(input_str)
            targets.append(answer_text)

        model_inputs = tokenizer(
            inputs,
            max_length=max_input_length,
            padding="max_length",
            truncation=True,
        )

        with tokenizer.as_target_tokenizer():
            labels = tokenizer(
                targets,
                max_length=max_target_length,
                padding="max_length",
                truncation=True,
            )

        model_inputs["labels"] = labels["input_ids"]
        return model_inputs
    return preprocess_function

In [ ]:
max_input_values = [256, 384, 512]
...
for max_len in max_input_values:
    ...

In [ ]:
from datasets import load_dataset

squad = load_dataset("rajpurkar/squad")
train_ds = squad["train"]
valid_ds = squad["validation"]

small_train = train_ds.select(range(5000))      # subset train
small_valid = valid_ds.select(range(1000))      # subset valid

print(len(small_train), len(small_valid))
print(small_train.column_names)

README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

plain_text/validation-00000-of-00001.par(…):   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

5000 1000
['id', 'title', 'context', 'question', 'answers']


In [ ]:
max_input_values = [256, 384, 512]

results_maxlen = []

for max_len in max_input_values:
    print(f"\n=== max_input_length = {max_len} ===")

    tokenizer = T5TokenizerFast.from_pretrained("t5-base")
    preprocess_function = build_preprocess_fn(tokenizer, max_input_length=max_len)

    # tokenisasi ulang subset agar cepat
    tokenized_train = small_train.map(
        preprocess_function,
        batched=True,
        remove_columns=small_train.column_names,
    )
    tokenized_valid = small_valid.map(
        preprocess_function,
        batched=True,
        remove_columns=small_valid.column_names,
    )

    model = T5ForConditionalGeneration.from_pretrained("t5-base").to(device)
    data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

    training_args = Seq2SeqTrainingArguments(
        output_dir=f"./t5-squad-maxlen-{max_len}",
        do_train=True,
        do_eval=True,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        learning_rate=3e-4,
        num_train_epochs=1,
        weight_decay=0.01,
        logging_steps=100,
        save_total_limit=1,
        predict_with_generate=True,
        fp16=torch.cuda.is_available(),
    )

    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_valid,
        tokenizer=tokenizer,
        data_collator=data_collator,
    )

    trainer.train()
    trainer.save_model(f"./t5-squad-maxlen-{max_len}")
    tokenizer.save_pretrained(f"./t5-squad-maxlen-{max_len}")


=== max_input_length = 256 ===


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

/tmp/ipython-input-3030395755.py:41: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

...

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

/tmp/ipython-input-3030395755.py:41: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Step,Training Loss
100,1.448300
200,0.073600
300,0.069100
400,0.064600
500,0.064100
600,0.063100
700,0.065000
800,0.060300
900,0.053700
1000,0.051400


In [ ]:
out_dir = f"/content/drive/MyDrive/T5_QA_SQuAD_epochs/epochs_{ep}"
trainer.save_model(out_dir)
tokenizer.save_pretrained(out_dir)

('/content/drive/MyDrive/T5_QA_SQuAD_epochs/epochs_1/tokenizer_config.json',
 '/content/drive/MyDrive/T5_QA_SQuAD_epochs/epochs_1/special_tokens_map.json',
 '/content/drive/MyDrive/T5_QA_SQuAD_epochs/epochs_1/spiece.model',
 '/content/drive/MyDrive/T5_QA_SQuAD_epochs/epochs_1/added_tokens.json',
 '/content/drive/MyDrive/T5_QA_SQuAD_epochs/epochs_1/tokenizer.json')

# Step 7: Basic Evaluation & **Inference**

In [ ]:
def answer_question(context, question, max_output_length=32):
    model.eval()
    input_text = f"question: {question} context: {context}"
    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        truncation=True,
        max_length=512,
    ).to(device)

    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_length=max_output_length,
            num_beams=4,
        )

    answer = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return answer

In [ ]:
from datasets import load_dataset
from random import randrange

squad = load_dataset("rajpurkar/squad")
val_dataset = squad["validation"]

for _ in range(3):
    idx = randrange(len(val_dataset))
    ex = val_dataset[idx]
    pred = answer_question(ex["context"], ex["question"])
    print("="*80)
    print("Question:", ex["question"])
    print("Gold   :", ex["answers"]["text"][0])
    print("Pred   :", pred)

Question: How might gravity effects be observed differently according to Newton?
Gold   : at larger distances.
Pred   : larger distances
Question: What is the prize offered for finding a solution to P=NP?
Gold   : $1,000,000
Pred   : US$1,000,000
Question: What color were the Bronco's uniforms in Super Bowl 50?
Gold   : white
Pred   : white


In [ ]:
dir_ep = f"/content/drive/MyDrive/T5_QA_SQuAD_epochs/epochs_{ep}"
tokenizer = T5TokenizerFast.from_pretrained(dir_ep)
model = T5ForConditionalGeneration.from_pretrained(dir_ep).to(device)

In [ ]:
dir_ep = f"/content/drive/MyDrive/t5-squad-epochs-{ep}"

In [ ]:
import os, glob

print(glob.glob("./t5-squad-epochs-*"))
print(glob.glob("/content/drive/MyDrive/*"))

[]
['/content/drive/MyDrive/IMG-20210302-WA0008.jpg', '/content/drive/MyDrive/IMG-20210227-WA0019.jpg', '/content/drive/MyDrive/IMG-20210302-WA0007.jpg', '/content/drive/MyDrive/IMG-20210227-WA0018.jpg', '/content/drive/MyDrive/IMG-20210302-WA0006.jpg', '/content/drive/MyDrive/IMG-20210302-WA0005.jpg', '/content/drive/MyDrive/IMG-20210302-WA0015.jpg', '/content/drive/MyDrive/IMG-20210302-WA0004.jpg', '/content/drive/MyDrive/IMG-20210302-WA0014.jpg', '/content/drive/MyDrive/IMG-20210302-WA0003.jpg', '/content/drive/MyDrive/IMG-20210302-WA0002.jpg', '/content/drive/MyDrive/IMG-20210302-WA0013.jpg', '/content/drive/MyDrive/IMG-20210302-WA0012.jpg', '/content/drive/MyDrive/IMG-20210227-WA0017.jpg', '/content/drive/MyDrive/IMG-20210302-WA0011.jpg', '/content/drive/MyDrive/IMG-20210227-WA0016.jpg', '/content/drive/MyDrive/IMG-20210302-WA0010.jpg', '/content/drive/MyDrive/IMG-20210227-WA0015.jpg', '/content/drive/MyDrive/IMG-20210302-WA0009.jpg', '/content/drive/MyDrive/2021_05_21_00_53_01 (1

In [ ]:
import glob

print(glob.glob("/content/drive/MyDrive/T5_QA_SQUAD_epochs/*"))

[]


# Step 8 : hitung metrik Exact Match (EM) dan F1 SQuAD di validation set

In [ ]:
import evaluate

squad_metric = evaluate.load("squad")

In [ ]:
from datasets import load_dataset

raw_squad = load_dataset("rajpurkar/squad")
val_ds = raw_squad["validation"]

num_eval_samples = 500  # bisa dinaikkan nanti
val_subset = val_ds.select(range(num_eval_samples))

In [ ]:
def generate_answers_batch(examples, max_input_length=512, max_output_length=32):
    inputs = [
        f"question: {q} context: {c}"
        for q, c in zip(examples["question"], examples["context"])
    ]
    model_inputs = tokenizer(
        inputs,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=max_input_length,
    ).to(device)

    with torch.no_grad():
        output_ids = model.generate(
            **model_inputs,
            max_length=max_output_length,
            num_beams=4,
        )

    preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
    return preds

In [ ]:
all_predictions = []
all_references = []

for start in range(0, len(val_subset), 16):  # batch size 16
    end = start + 16
    batch = val_subset[start:end]          # ini dict-of-lists
    preds = generate_answers_batch(batch)

    # iterasi pakai index
    for i, pred in enumerate(preds):
        ex_id = batch["id"][i]
        ex_answers = batch["answers"][i]

        all_predictions.append({
            "id": ex_id,
            "prediction_text": pred,
        })
        all_references.append({
            "id": ex_id,
            "answers": ex_answers,
        })

In [ ]:
results = squad_metric.compute(
    predictions=all_predictions,
    references=all_references,
)

print(results)

{'exact_match': 83.2, 'f1': 89.29111111111116}


In [ ]:
def evaluate_model(model_dir, val_subset):
    from transformers import T5ForConditionalGeneration, T5TokenizerFast

    tokenizer = T5TokenizerFast.from_pretrained(model_dir)
    model = T5ForConditionalGeneration.from_pretrained(model_dir).to(device)
    model.eval()

    all_predictions = []
    all_references = []

    for start in range(0, len(val_subset), 16):
        end = start + 16
        batch = val_subset[start:end]
        preds = generate_answers_batch(batch)  # fungsi yang sudah kamu punya

        for i, pred in enumerate(preds):
            ex_id = batch["id"][i]
            ex_answers = batch["answers"][i]
            all_predictions.append({"id": ex_id, "prediction_text": pred})
            all_references.append({"id": ex_id, "answers": ex_answers})

    metrics = squad_metric.compute(
        predictions=all_predictions,
        references=all_references,
    )
    return metrics["exact_match"], metrics["f1"]

# Step 9 : error analysis dan analisis kualitatif jawaban model di SQuAD.

In [ ]:
def collect_examples_with_scores(num_samples=100):
    examples = []
    subset = val_ds.select(range(num_samples))  # val_ds = raw_squad["validation"]

    preds = generate_answers_batch(subset)

    for i, (ex, pred) in enumerate(zip(subset, preds)):
        gold_answers = ex["answers"]["text"]

        # buat format metric untuk satu contoh
        prediction = [{"id": ex["id"], "prediction_text": pred}]
        reference = [{"id": ex["id"], "answers": ex["answers"]}]
        scores = squad_metric.compute(
            predictions=prediction,
            references=reference,
        )

        examples.append({
            "id": ex["id"],
            "question": ex["question"],
            "context": ex["context"],
            "gold_answers": gold_answers,
            "pred": pred,
            "exact_match": scores["exact_match"],
            "f1": scores["f1"],
        })
    return examples

examples = collect_examples_with_scores(num_samples=200)

In [ ]:
# sort dari F1 terendah
examples_sorted = sorted(examples, key=lambda x: x["f1"])

for ex in examples_sorted[:5]:
    print("="*80)
    print("ID       :", ex["id"])
    print("Question :", ex["question"])
    print("Gold     :", ex["gold_answers"])
    print("Pred     :", ex["pred"])
    print("EM / F1  :", ex["exact_match"], "/", ex["f1"])

ID       : 56be8e613aeaaa14008c90d1
Question : What was the theme of Super Bowl 50?
Gold     : ['"golden anniversary"', 'gold-themed', '"golden anniversary']
Pred     : gold
EM / F1  : 0.0 / 0.0
ID       : 56beace93aeaaa14008c91e1
Question : What city did Super Bowl 50 take place in?
Gold     : ['Santa Clara', 'Santa Clara', 'Santa Clara']
Pred     : San Francisco Bay Area
EM / F1  : 0.0 / 0.0
ID       : 56bf10f43aeaaa14008c94fe
Question : What city did Super Bowl 50 take place in?
Gold     : ['Santa Clara', 'Santa Clara', 'Santa Clara']
Pred     : San Francisco Bay Area
EM / F1  : 0.0 / 0.0
ID       : 56d2045de7d4791d009025f5
Question : How many times have the Panthers been in the Super Bowl?
Gold     : ['2', 'second', 'second']
Pred     : eight
EM / F1  : 0.0 / 0.0
ID       : 56d2045de7d4791d009025f6
Question : Who did Denver beat in the AFC championship?
Gold     : ['New England Patriots', 'the New England Patriots', 'New England Patriots']
Pred     : the Arizona Cardinals
EM / F1  

# Step 10 inference demo

In [ ]:
def qa_pipeline(context, question, max_output_length=32, num_beams=4):
    model.eval()
    input_text = f"question: {question} context: {context}"
    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        truncation=True,
        max_length=512,
    ).to(device)

    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_length=max_output_length,
            num_beams=num_beams,
        )

    answer = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return answer

In [ ]:
custom_context = """
Bandung is the capital city of West Java province in Indonesia.
It is known for its universities, cool climate, and surrounding volcanoes and tea plantations.
"""

custom_question = "What is Bandung the capital city of?"

print("Question:", custom_question)
print("Answer  :", qa_pipeline(custom_context, custom_question))

Question: What is Bandung the capital city of?
Answer  : West Java province


Step 11 : Ablation Study (num_beams)

In [ ]:
def generate_answers_batch(examples, max_input_length=512, max_output_length=32, num_beams=4):
    inputs = [
        f"question: {q} context: {c}"
        for q, c in zip(examples["question"], examples["context"])
    ]
    model_inputs = tokenizer(
        inputs,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=max_input_length,
    ).to(device)

    model.eval()
    with torch.no_grad():
        output_ids = model.generate(
            **model_inputs,
            max_length=max_output_length,
            num_beams=num_beams,
        )

    preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
    return preds

In [ ]:
from datasets import load_dataset

raw_squad = load_dataset("rajpurkar/squad")
val_ds = raw_squad["validation"]

num_eval_samples = 500   # sama seperti sebelumnya
val_subset = val_ds.select(range(num_eval_samples))

In [ ]:
import time

beam_values = [1, 2, 4, 8]
results_beams = []

for nb in beam_values:
    print(f"\n=== Evaluating with num_beams = {nb} ===")
    start_time = time.time()

    all_predictions = []
    all_references = []

    for start in range(0, len(val_subset), 16):  # batch size 16
        end = start + 16
        batch = val_subset[start:end]           # dict-of-lists
        preds = generate_answers_batch(batch, num_beams=nb)

        for i, pred in enumerate(preds):
            ex_id = batch["id"][i]
            ex_answers = batch["answers"][i]

            all_predictions.append({
                "id": ex_id,
                "prediction_text": pred,
            })
            all_references.append({
                "id": ex_id,
                "answers": ex_answers,
            })

    metrics = squad_metric.compute(
        predictions=all_predictions,
        references=all_references,
    )
    elapsed = time.time() - start_time

    results_beams.append({
        "num_beams": nb,
        "exact_match": metrics["exact_match"],
        "f1": metrics["f1"],
        "time_sec": elapsed,
    })

    print(f"EM: {metrics['exact_match']:.2f}, F1: {metrics['f1']:.2f}, Time: {elapsed:.1f} s")


=== Evaluating with num_beams = 1 ===
EM: 84.20, F1: 89.86, Time: 11.4 s

=== Evaluating with num_beams = 2 ===
EM: 83.20, F1: 89.18, Time: 16.1 s

=== Evaluating with num_beams = 4 ===
EM: 83.20, F1: 89.29, Time: 29.1 s

=== Evaluating with num_beams = 8 ===
EM: 82.80, F1: 88.89, Time: 55.4 s


In [ ]:
print("\nSummary num_beams ablation:")
for r in results_beams:
    print(
        f"num_beams={r['num_beams']}: "
        f"EM={r['exact_match']:.2f}, "
        f"F1={r['f1']:.2f}, "
        f"time={r['time_sec']:.1f}s"
    )


Summary num_beams ablation:
num_beams=1: EM=84.20, F1=89.86, time=12.7s
num_beams=2: EM=83.20, F1=89.18, time=16.1s
num_beams=4: EM=83.20, F1=89.29, time=27.3s
num_beams=8: EM=82.60, F1=88.69, time=42.9s


In [ ]:
ablation_epochs = [
    {"epochs": 1, "em": None, "f1": None},
    {"epochs": 2, "em": None, "f1": None},
    {"epochs": 3, "em": None, "f1": None},
]

for r in ablation_epochs:
    print("Epoch", r["epochs"], "EM:", r["em"], "F1:", r["f1"])


Epoch 1 EM: None F1: None
Epoch 2 EM: None F1: None
Epoch 3 EM: None F1: None


In [ ]:
ablation_maxlen = [
    {"max_len": 256, "em": ..., "f1": ...},
    {"max_len": 384, "em": ..., "f1": ...},
    {"max_len": 512, "em": ..., "f1": ...},
]

In [ ]:
import torch
torch.cuda.empty_cache()
